# Python notebook for post-processing apical responses.
# Create image stack movie with regions plot.
# --- UNDER DEVELOPMENT ---
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.

In [5]:
%matplotlib widget

import glob
import ipywidgets as widgets
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from skimage import io



#### User editable lookup table for image plot colors.
For a list of predefied colormaps refer to:
https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html \
To create your own custom colormap refer to:
https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html


In [6]:
# look up table for image plot color
#   e.g. coolwarm, jet, plasma, gray
lut = mpl.cm.gray


In [7]:
# global variables
image_sel = ""
results_sel = ""
regions_sel = []

# create image files widget
image_files = sorted([f.split('/')[-1] for f in glob.glob("../image_stacks/*.tif", recursive=False)], key=str.casefold)
image_widget = widgets.Select(options=image_files, description='Image stack:', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# create results directory widget
result_dirs = sorted([f.split('/')[-2] for f in glob.glob("../results/*/", recursive=False)], key=str.casefold)
results_widget = widgets.Select(options=result_dirs, description='Results dir:', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# create regions widget
regions_widget = widgets.SelectMultiple(options=[], description='Regions:', 
                            disabled=False, layout=widgets.Layout(width='150px'))

# update the regions based on the results directory selection
def update_regions(*args):
  region_files = os.listdir("../results/" + results_widget.value)
  region_files = [f for f in region_files if 'apical_region' in f and '.csv' in f][:-1]
  regions = [f.split('-')[0].split('_')[-1] for f in region_files]
  regions = sorted([int(r) for r in regions])
  regions_widget.options = regions

# results directory change callback
results_widget.observe(update_regions, 'value')

# display and respond to the widgets
def f(w1, w2, w3):
  global image_sel, results_sel, regions_sel
  image_sel = image_widget.value
  results_sel = results_widget.value
  regions_sel = list(regions_widget.value)
display(widgets.interactive(f, w1=image_widget, w2=results_widget, w3=regions_widget))

interactive(children=(Select(description='Image stack:', layout=Layout(width='400px'), options=('lessmovement3…

In [4]:
print(image_sel)
print(results_sel)
print(regions_sel)

Mistgcamp-3_0003.tif
Mistgcamp-320200527-212243
[1, 3]


In [3]:
backend = mpl.get_backend()
mpl.use("Agg")  # don't display the animation

image_file = "../image_stacks/Mistgcamp-3_0002.tif"
image_bits = 10
A0 = io.imread(image_file)
A = np.float32(A0/(2.0**image_bits))

data_file = "../results/Mistgcamp-320200527-212243/apical_region_2-mask5Hz-stimALL.csv"
data_ind = 3  # the data index to animate
D = np.genfromtxt(data_file, delimiter=',')

widths = [1]
heights = [1, 0.2]
gs_kw = dict(width_ratios=widths, height_ratios=heights)
fig,ax = plt.subplots(nrows=2, ncols=1, 
                      constrained_layout=True, gridspec_kw=gs_kw,
                      figsize = [6, 8], dpi=100)
# adjust_subplots???
ax[0].set_xticks([])
ax[0].set_yticks([])
im = ax[0].imshow(A[0], norm=None, cmap=lut)
ax[1].set_xlim(0,10.0*np.ceil(np.amax(D[:,0]/10.0)))
ax[1].set_ylim(0,100.0*np.ceil(np.amax(D[:,data_ind+1]/100.0)))
X = []
Y = []
line, = ax[1].plot(D[0,0], D[0,data_ind])

def animation_frame(i):
  X.append(D[i,0])
  Y.append(D[i,data_ind])
  line.set_xdata(X)
  line.set_ydata(Y)
  im.set_data(A[i])
  return

animation = FuncAnimation(fig, animation_frame, blit=False, repeat=False,
                          frames=D.shape[0], interval=np.int(1000.0*D[1,0]))
animation.save("../image_stacks/tempB.mp4")
mpl.use(backend)